# In this notebook, we show the code for analysis.

We would like to reconstruct Fig. 12 in [Wenegrat et al. (2018)](https://journals.ametsoc.org/doi/full/10.1175/JPO-D-17-0219.1) to see the relative importance of surface forcing on mode water formation in our submesoscale permitting North Atlantic simulation with tidal forcing (eNATL60). The spatial resolution is 1/60$^\circ$ with 300 vertical layers allowing for well resolved eddies. The sea-surface data from a run without tidal forcing are available on [Pangeo](https://catalog.pangeo.io/browse/master/ocean/MEOM_NEMO/).

The contributions due to turbulent thermal wind ($J_\text{TTW}$), surface wind kinematic forcing ($J_\text{wind}$) and buoyancy forcing ($J^\text{buoy}_\text{D}$) are diagnosed as: 
$$J_\text{TTW} \simeq -0.05H_\text{ML}|\overline{\nabla_\text{h}b}^z|^2,$$

$$J_\text{wind} \simeq f\frac{\text{EBF}}{H_\text{ML}} = \frac{({\bf \tau}^w\times {\bf k})\cdot \overline{\nabla_\text{h}b}^z}{\rho H_\text{ML}} = \frac{\tau^y\overline{b_x}^z - \tau^x\overline{b_y}^z}{\rho H_\text{ML}}$$
where $\text{EBF} = \frac{1}{\rho f}({\bf \tau}^w\times {\bf k})\cdot\overline{\nabla_\text{h}b}^z$ is the Ekman buoyancy flux, and
$$J^\text{buoy}_\text{D} \simeq f c_s \frac{B_0}{H_\text{ML}}$$
where $c_s = 1.2$ comes from the assumption that that the turbulent vertical buoyancy flux divergence will remain similar, in some area-integrated sense, to classic upright convection ([Wenegrat et al., 2018](https://journals.ametsoc.org/doi/full/10.1175/JPO-D-17-0219.1)).
The surface buoyancy flux is ([Buckingham et al., 2019](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2019MS001801)):
$$B_0 = \alpha g\frac{Q_0}{\rho_0 c_p} + \beta g (E-P) S_0$$
where $Q_0, E-P$ and $S_0$ are the surface net heat, fresh-water flux and salinity respectively. 

The fluxes are integrated in time only when and where the mode-water isopycnal outcrops at the surface. 
For simplicity and with prior knowledge that density is temperature dominated in the Gulf Stream region, we will define the mode water as waters with $18 \pm 1 ^\circ\text{C}$.

In [1]:
from dask.distributed import Client, LocalCluster


###################################
# We start an interactive session with 12 dask workers.
# The power of dask is that setting up a cluster for parallel computation
# is mostly done in this cell, which allows us to focus on the science.
###################################
cluster = LocalCluster()
cluster.scale(12)

client = Client(cluster)
client

/scratch/cnt0024/hmg2840/tuchida/condapack/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:37182 Dashboard: http://127.0.0.1:43250/status,Cluster Workers: 12 Cores: 84 Memory: 405.28 GB


In [2]:
import xarray as xr
import numpy as np
import xscale
import gsw
import os.path as op
from xhistogram.xarray import histogram as xhist

from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
ddir = '/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLBT02-S/'
xtra = '/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLBT02X-S'
scratch = '/scratch/cnt0024/hmg2840/tuchida/temp'

In [4]:
ys,ye,xs,xe = (1400,2800,950,2550)
g = 9.81
cp = 4e3
cs = 1.2
zchunk = 10
xchunk = -1
ychunk = -1
tchunk = -1

gdepw = xr.open_dataset(op.join(scratch,'gdepw_eNATL60.nc')
                       ).gdepw.sel(y=slice(ys,ye),x=slice(xs,xe)
                                  ).chunk({'y':ychunk,'x':xchunk})
dsmask = xr.open_dataset(op.join(ddir,'../eNATL60-I/mesh_mask_eNATL60_3.6.nc'), 
                         chunks={'z':zchunk,'y':ychunk,'x':xchunk}
                        ).isel(y=slice(ys,ye),x=slice(xs,xe)).isel(t=0)
f = xr.apply_ufunc(gsw.f, dsmask.nav_lat, dask='parallelized', output_dtypes=[float,])

In [5]:
# days = np.concatenate((np.arange(12,32,dtype=int), np.arange(1,6,dtype=int)))
days = np.arange(6,30, dtype=int)
dirs = 1393201
print(days,dirs)

[ 6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29] 1393201


### We iterate over each daily file, which stores 24 timesteps of hourly averaged data.

In [6]:
month = 10
year = 2010
istart = -3

for i in days[istart:]:
    j = month
    l = j
    m = month+0
    
    if i < days[0]:
        l = m
            
###################################
# We open the files necessary for analysis.
# The files are chunked along each dimension where dask automatically
# executes the computation on each chunk in parallel.
# At this point, no data is loaded onto memory yet and 
# they are symbolic stake holders up until they are called upon to compute.
###################################
    dsT = xr.open_dataset(op.join(xtra,'%08d-%08d/eNATL60-BLBT02X_1h_%4d%02d%02d_%4d%02d%02d_gridT_%4d%02d%02d-%4d%02d%02d.nc' 
                                  % (dirs,int(dirs+10800*len(days)*.2-1),
                                     year,j,days[0],year,m,days[-1],
                                     year,l,i,year,l,i)),
                          chunks={'time_counter':tchunk,'deptht':zchunk,'y':ychunk,'x':xchunk}
                         ).sel(deptht=slice(None,810))
    dsS = xr.open_dataset(op.join(xtra,'%08d-%08d/eNATL60-BLBT02X_1h_%4d%02d%02d_%4d%02d%02d_gridS_%4d%02d%02d-%4d%02d%02d.nc' 
                                  % (dirs,int(dirs+10800*len(days)*.2-1),
                                     year,j,days[0],year,m,days[-1],
                                     year,l,i,year,l,i)),
                          chunks={'time_counter':tchunk,'deptht':zchunk,'y':ychunk,'x':xchunk}
                         ).sel(deptht=slice(None,810))
    dsflx = xr.open_dataset(op.join(xtra,'%08d-%08d/eNATL60-BLBT02X_1h_%4d%02d%02d_%4d%02d%02d_flxT_%4d%02d%02d-%4d%02d%02d.nc'
                                    % (dirs,int(dirs+10800*len(days)*.2-1),
                                       year,j,days[0],year,m,days[-1],
                                       year,l,i,year,l,i)),
                            chunks={'time_counter':tchunk,'y':ychunk,'x':xchunk}
                           )
    dsH = xr.open_dataset(op.join(xtra,'%08d-%08d/eNATL60-BLBT02X_1h_%4d%02d%02d_%4d%02d%02d_gridT-2D_%4d%02d%02d-%4d%02d%02d.nc' 
                                  % (dirs,int(dirs+10800*len(days)*.2-1),
                                     year,j,days[0],year,m,days[-1],
                                     year,l,i,year,l,i)),
                          chunks={'time_counter':tchunk,'y':ychunk,'x':xchunk}
                         )
    dsx = xr.open_dataset(op.join(xtra,'%08d-%08d/eNATL60-BLBT02X_1h_%4d%02d%02d_%4d%02d%02d_gridU-2D_%4d%02d%02d-%4d%02d%02d.nc' 
                                  % (dirs,int(dirs+10800*len(days)*.2-1),
                                     year,j,days[0],year,m,days[-1],
                                     year,l,i,year,l,i)),
                          chunks={'time_counter':tchunk,'y':ychunk,'x':xchunk}
                         )
    dsy = xr.open_dataset(op.join(xtra,'%08d-%08d/eNATL60-BLBT02X_1h_%4d%02d%02d_%4d%02d%02d_gridV-2D_%4d%02d%02d-%4d%02d%02d.nc' 
                                  % (dirs,int(dirs+10800*len(days)*.2-1),
                                     year,j,days[0],year,m,days[-1],
                                     year,l,i,year,l,i)),
                          chunks={'time_counter':tchunk,'y':ychunk,'x':xchunk}
                         )
        
        
    if i == days[istart]:
        maskT = dsmask.tmask.isel(z=slice(None,len(dsT.deptht)))
        maskU = dsmask.umask.isel(z=0)
        maskV = dsmask.vmask.isel(z=0)
        
    for tt in range(len(dsflx.time_counter)):
        CT = dsT.votemper.isel(y=slice(ys,ye),
                               x=slice(xs,xe)).isel(time_counter=tt).where(xr.DataArray(maskT.data, 
                                                                                        dims=['deptht','y','x']) != 0.)
        SA = dsS.vosaline.isel(y=slice(ys,ye),
                               x=slice(xs,xe)).isel(time_counter=tt).where(xr.DataArray(maskT.data, 
                                                                                        dims=['deptht','y','x']) != 0.)
        ssh = dsH.sossheig.isel(y=slice(ys,ye),
                                x=slice(xs,xe)).isel(time_counter=tt).where(xr.DataArray(maskT.isel(z=0).data, 
                                                                                         dims=['y','x']) != 0.)
        taux = dsx.sozotaux.isel(y=slice(ys,ye),
                                 x=slice(xs,xe)).isel(time_counter=tt).where(xr.DataArray(maskU.data, 
                                                                                          dims=['y','x']) != 0.)
        tauy = dsy.sometauy.isel(y=slice(ys,ye),
                                 x=slice(xs,xe)).isel(time_counter=tt).where(xr.DataArray(maskV.data, 
                                                                                          dims=['y','x']) != 0.)

###################################
# We compute in parallel the potential density (sig0), mixed-layer depth (MLD),
# surface buoyancy forcing (Bo), and the MLD averaged horizontal gradients of buoyancy.
# Data is still not loaded into memory.
###################################
        sig0 = xr.apply_ufunc(gsw.sigma0, SA, CT, 
                              dask='parallelized', output_dtypes=[float,])
        buoy = -g*sig0*1e-3
        z10 = 6
        nMLD = z10 + np.abs((sig0.isel(deptht=slice(z10,None)).fillna(999.)
                             - sig0.isel(deptht=z10).fillna(999.)
                            ) - 0.03).argmin(dim='deptht',skipna=True)
        e3w = dsmask.e3w_0 * (1+ssh*gdepw**-1)
        e3t = dsmask.e3t_0 * (1+ssh*gdepw**-1)

        MLD = e3t.fillna(0.).where(e3t.z <= nMLD.fillna(0.).compute()
                                  ).sum('z',skipna=True)

        alpha = xr.apply_ufunc(gsw.alpha, SA.isel(deptht=0), CT.isel(deptht=0), 0.,
                               dask='parallelized', output_dtypes=['float',])
        beta = xr.apply_ufunc(gsw.beta, SA.isel(deptht=0), CT.isel(deptht=0), 0.,
                              dask='parallelized', output_dtypes=['float',])
        Bo = g*(-alpha * dsflx.qt_oce.isel(time_counter=tt).isel(y=slice(ys,ye),x=slice(xs,xe)) 
                * ((sig0.isel(deptht=0)+1e3)*cp)**-1
                + beta * dsflx.sowaflup.isel(time_counter=tt).isel(y=slice(ys,ye),x=slice(xs,xe))
                  * SA.isel(deptht=0)
               )    # Positive values defined as destablizing conditions
        
        bx = (buoy.isel(x=slice(1,None))
              + buoy.isel(x=slice(None,-1)).data
             ) * .5
        by = (buoy.isel(y=slice(1,None))
              + buoy.isel(y=slice(None,-1)).data
             ) * .5
        dbx = bx.diff(dim='x') * dsmask.e1u.isel(x=slice(1,-1))**-1
        dby = by.diff(dim='y') * dsmask.e2v.isel(y=slice(1,-1))**-1


        dbx = (dbx.fillna(0.) * xr.DataArray(e3t.isel(z=slice(len(dsT.deptht))).isel(x=slice(1,-1)).data,
                                             dims=['deptht','y','x'])
              ).where(dbx.deptht <= MLD.isel(x=slice(1,-1))
                     ).sum('deptht',skipna=True) * MLD.isel(x=slice(1,-1))**-1
        dby = (dby.fillna(0.) * xr.DataArray(e3t.isel(z=slice(len(dsT.deptht))).isel(y=slice(1,-1)).data,
                                             dims=['deptht','y','x'])
              ).where(dby.deptht <= MLD.isel(y=slice(1,-1))
                     ).sum('deptht',skipna=True) * MLD.isel(y=slice(1,-1))**-1
        db2 = dbx.isel(y=slice(1,-1))**2 + dby.isel(x=slice(1,-1))**2


###################################
# With the command .load() and .compute(), the data are loaded for the first time into memory.
# The "J" fluxes are computed and saved below.
###################################
        sst = CT.isel(deptht=0).load()

        if tt == 0:
            J_d = (Bo*f*cs*MLD**-1).compute().where(sst<19).where(sst>17)
            J_ttw = -0.05*(MLD.isel(y=slice(1,-1),x=slice(1,-1)) * db2
                          ).compute().where(sst.isel(y=slice(1,-1),x=slice(1,-1))<19
                                           ).where(sst.isel(y=slice(1,-1),x=slice(1,-1))>17)
            J_wind = ((.5*(tauy.isel(y=slice(1,None))+tauy.isel(y=slice(None,-1))).isel(y=slice(1,None),x=slice(1,-1)) 
                       * dbx.isel(y=slice(1,-1)) 
                       - .5*(taux.isel(x=slice(1,None))+taux.isel(x=slice(None,-1))).isel(y=slice(1,-1),x=slice(1,None)) 
                         * dby.isel(x=slice(1,-1))
                      ) * ((sig0.fillna(0.)+1e3) * xr.DataArray(e3t[:len(dsT.deptht)].fillna(0.).data, 
                                                                dims=['deptht','y','x']
                                                               )
                          ).where(sig0.deptht <= MLD).isel(y=slice(1,-1),x=slice(1,-1)).sum('deptht',skipna=True)**-1
                     ).compute().where(sst.isel(y=slice(1,-1),x=slice(1,-1))<19
                                      ).where(sst.isel(y=slice(1,-1),x=slice(1,-1))>17)
        else:
            J_d = xr.concat([J_d, (Bo*f*cs*MLD**-1).compute().where(sst<19).where(sst>17)], 
                            'time_counter')
            J_ttw = xr.concat([J_ttw, -0.05*(MLD.isel(y=slice(1,-1),x=slice(1,-1)) * db2
                                            ).compute().where(sst.isel(y=slice(1,-1),x=slice(1,-1))<19
                                                             ).where(sst.isel(y=slice(1,-1),x=slice(1,-1))>17)],
                              'time_counter')
            J_wind = xr.concat([J_wind, ((.5*(tauy.isel(y=slice(1,None))+tauy.isel(y=slice(None,-1))).isel(y=slice(1,None),
                                                                                                           x=slice(1,-1)) 
                                          * dbx.isel(y=slice(1,-1)) 
                                          - .5*(taux.isel(x=slice(1,None))+taux.isel(x=slice(None,-1))).isel(y=slice(1,-1),
                                                                                                             x=slice(1,None)) 
                                          * dby.isel(x=slice(1,-1))
                                         ) * ((sig0.fillna(0.)+1e3) * xr.DataArray(e3t[:len(dsT.deptht)].fillna(0.).data, 
                                                                                   dims=['deptht','y','x']
                                                                                  )
                                             ).where(sig0.deptht <= MLD).isel(y=slice(1,-1),
                                                                              x=slice(1,-1)).sum('deptht',skipna=True)**-1
                                        ).compute().where(sst.isel(y=slice(1,-1),x=slice(1,-1))<19
                                                         ).where(sst.isel(y=slice(1,-1),x=slice(1,-1))>17)],
                                'time_counter')
        
        
    dsT.close()
    dsS.close()
    dsH.close()
    dsflx.close()
    dsx.close()
    dsy.close() 
    
###################################
# The "J" fluxes are saved as netcdf files.
###################################
    dsave = J_d.isel(y=slice(1,-1),x=slice(1,-1)).drop_vars('time_centered').to_dataset(name='J_D')
    dsave['J_TTW'] = J_ttw.drop_vars('time_centered')
    dsave['J_wind'] = J_wind.drop_vars('time_centered')
    dsave.to_netcdf(op.join(scratch,'GulfStream/Jflux_%4d-%02d-%02d.nc' 
                                % (year,l,i)))
    dsave.close()
    print(r"Day: %02d-%02d" % (l,i))

Day: 10-27
Day: 10-28
Day: 10-29
